# Toronto COVID cases by neighborhood

##  Introduction/Business Problem

In the course of the COVID pandemic, all goverments implemented several rules and restrictions in order to mitigate the outbreaks of the disease, these restrictions were usually defined based on the type of business and industry of the venues.

In this notebook I will try to rate the efficacy of the restrictions put in place by the goverment in the city of Toronto, by analizying the data of the total COVID cases and the type of venues, all segmented by Postal Code and neighborhood.

The target audience would be all the involved goverment dependenciees (Commerce, Health, City Planning) which could determine if the imposed restrictions did work and if any of these should be kept in place. Also to define a workbook for future pandemics. 

## Data

**COVID-19 Cases in Toronto**

For the COVID cases in Toronto I will use the dataset provided by the Toronto city goverment in their Open Data portal:
https://open.toronto.ca/dataset/covid-19-cases-in-toronto/

An example of the dataset is below, wihch includes all the requiered features for the analysis (age, postal code, neighborhood)

_id  | Assigned_ID |Outbreak Associated | Age Group | Neighbourhood Name | FSA | Source of Infection | Classification | Episode Date | Reported Date | Client Gender | Outcome | Currently Hospitalized | Currently in ICU | Currently Intubated | Ever Hospitalized | Ever in ICU | Ever Intubated
-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----
3326211 |99865 |Sporadic| 19 and younger| Thorncliffe Park| M4H| Household Contact| CONFIRMED| 2021-02-24| 2021-02-25| FEMALE| RESOLVED| No| No| No |  No |  No |  No
3326212 |  99866 |  Sporadic |  40 to 49 Years |  Taylor-Massey |  M4C |  Household Contact |  CONFIRMED |  2021-02-24 |  2021-02-25 |  FEMALE |  RESOLVED |  No |  No |  No |  No |  No |  No 


**Toronto neighborhood data**

I will continue to use the neighborhood data from the wikipedia page which was used in the previous weeks:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Using this data and the geocoder library, I will create a dataset with the following features:

PostalCode | Borough | Neighborhood | Latitude | Longitude
-----|-----|-----|-----|-----
M3A | North York | Parkwoods | 43.75245 | -79.32991
M4A | North York | Victoria Village | 43.73057 | -79.31306

**Foursquare API**

I will use the Foursquare API to gather the data of venues by neighborhood and define the most popular categories in each neighborhood